In [ ]:
from main import *
train_sub = [1,2,4,5,6,7,8,9]
val_sub = [3]
test_sub = [3]

train_without_aug(train_sub, val_sub, test_sub)

/home/junyeobe/.local/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
2023-05-16 18:21:15.159511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 18:21:15.659121: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-16 18:21:27.979361: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 18:21:28.00071

Epoch 1/1000


2023-05-16 18:21:38.718727: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-05-16 18:21:40.945181: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-05-16 18:21:41.708407: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-16 18:21:42.085401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-16 18:21:42.152994: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f59ebb83070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-16 18:21:42.153031: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0

72/72 [==============================] - 27s 61ms/step - loss: 1.3729 - accuracy: 0.3062 - val_loss: 1.3680 - val_accuracy: 0.2639
Epoch 2/1000
72/72 [==============================] - 3s 47ms/step - loss: 1.2858 - accuracy: 0.4067 - val_loss: 1.3154 - val_accuracy: 0.3177
Epoch 3/1000
72/72 [==============================] - 3s 47ms/step - loss: 1.1864 - accuracy: 0.4876 - val_loss: 1.2088 - val_accuracy: 0.4358
Epoch 4/1000
72/72 [==============================] - 3s 49ms/step - loss: 1.1129 - accuracy: 0.5352 - val_loss: 1.1241 - val_accuracy: 0.4740
Epoch 5/1000
72/72 [==============================] - 4s 49ms/step - loss: 1.0445 - accuracy: 0.5738 - val_loss: 1.0585 - val_accuracy: 0.5503
Epoch 6/1000
72/72 [==============================] - 4s 49ms/step - loss: 0.9906 - accuracy: 0.6007 - val_loss: 0.8208 - val_accuracy: 0.6997
Epoch 7/1000
72/72 [==============================] - 4s 49ms/step - loss: 0.9374 - accuracy: 0.6274 - val_loss: 0.6994 - val_accuracy: 0.7448
Epoch 8/100

72/72 [==============================] - 3s 45ms/step - loss: 0.4386 - accuracy: 0.8351 - val_loss: 0.4823 - val_accuracy: 0.8056
Epoch 59/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.4212 - accuracy: 0.8448 - val_loss: 0.5209 - val_accuracy: 0.7934
Epoch 60/1000
72/72 [==============================] - 3s 42ms/step - loss: 0.4431 - accuracy: 0.8346 - val_loss: 0.4912 - val_accuracy: 0.8021
Epoch 61/1000
72/72 [==============================] - 3s 42ms/step - loss: 0.4262 - accuracy: 0.8438 - val_loss: 0.5540 - val_accuracy: 0.7882
Epoch 62/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.4132 - accuracy: 0.8403 - val_loss: 0.4992 - val_accuracy: 0.8142
Epoch 63/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.4354 - accuracy: 0.8301 - val_loss: 0.4711 - val_accuracy: 0.8281
Epoch 64/1000
72/72 [==============================] - 3s 45ms/step - loss: 0.4199 - accuracy: 0.8411 - val_loss: 0.4410 - val_accuracy: 0.8108
Epoch 

Epoch 115/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.3271 - accuracy: 0.8785 - val_loss: 0.5456 - val_accuracy: 0.7969
Epoch 116/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2902 - accuracy: 0.8969 - val_loss: 0.5010 - val_accuracy: 0.8073
Epoch 117/1000
72/72 [==============================] - 3s 47ms/step - loss: 0.2996 - accuracy: 0.8906 - val_loss: 0.5386 - val_accuracy: 0.7951
Epoch 118/1000
72/72 [==============================] - 3s 45ms/step - loss: 0.3250 - accuracy: 0.8845 - val_loss: 0.5091 - val_accuracy: 0.8003
Epoch 119/1000
72/72 [==============================] - 3s 45ms/step - loss: 0.3179 - accuracy: 0.8826 - val_loss: 0.5465 - val_accuracy: 0.7986
Epoch 120/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.3044 - accuracy: 0.8859 - val_loss: 0.5484 - val_accuracy: 0.7951
Epoch 121/1000
72/72 [==============================] - 3s 45ms/step - loss: 0.2906 - accuracy: 0.8904 - val_loss: 0.6051 - val_ac

72/72 [==============================] - 3s 43ms/step - loss: 0.2309 - accuracy: 0.9119 - val_loss: 0.5796 - val_accuracy: 0.7934
Epoch 172/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2401 - accuracy: 0.9123 - val_loss: 0.5617 - val_accuracy: 0.7847
Epoch 173/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2368 - accuracy: 0.9145 - val_loss: 0.6064 - val_accuracy: 0.7899
Epoch 174/1000
72/72 [==============================] - 3s 42ms/step - loss: 0.2355 - accuracy: 0.9151 - val_loss: 0.7121 - val_accuracy: 0.7552
Epoch 175/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.2517 - accuracy: 0.9106 - val_loss: 0.6781 - val_accuracy: 0.7639
Epoch 176/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2434 - accuracy: 0.9121 - val_loss: 0.7227 - val_accuracy: 0.7587
Epoch 177/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.2454 - accuracy: 0.9121 - val_loss: 0.5464 - val_accuracy: 0.8073


72/72 [==============================] - 3s 44ms/step - loss: 0.1979 - accuracy: 0.9284 - val_loss: 0.6454 - val_accuracy: 0.7691
Epoch 228/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2054 - accuracy: 0.9297 - val_loss: 0.5508 - val_accuracy: 0.7969
Epoch 229/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.2291 - accuracy: 0.9149 - val_loss: 0.7298 - val_accuracy: 0.7743
Epoch 230/1000
72/72 [==============================] - 3s 46ms/step - loss: 0.2056 - accuracy: 0.9273 - val_loss: 0.6550 - val_accuracy: 0.7778
Epoch 231/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2111 - accuracy: 0.9251 - val_loss: 0.6110 - val_accuracy: 0.7865
Epoch 232/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.2024 - accuracy: 0.9288 - val_loss: 0.5245 - val_accuracy: 0.8229
Epoch 233/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.2058 - accuracy: 0.9227 - val_loss: 0.6921 - val_accuracy: 0.7795


72/72 [==============================] - 3s 44ms/step - loss: 0.1845 - accuracy: 0.9336 - val_loss: 0.8217 - val_accuracy: 0.7535
Epoch 284/1000
72/72 [==============================] - 3s 44ms/step - loss: 0.1867 - accuracy: 0.9295 - val_loss: 0.6377 - val_accuracy: 0.7812
Epoch 285/1000
72/72 [==============================] - 3s 45ms/step - loss: 0.1714 - accuracy: 0.9371 - val_loss: 0.7225 - val_accuracy: 0.7656
Epoch 286/1000
72/72 [==============================] - 3s 43ms/step - loss: 0.1787 - accuracy: 0.9368 - val_loss: 0.6258 - val_accuracy: 0.8108
Epoch 287/1000
72/72 [==============================] - 3s 47ms/step - loss: 0.1913 - accuracy: 0.9301 - val_loss: 0.6920 - val_accuracy: 0.7812
Epoch 288/1000
72/72 [==============================] - 3s 48ms/step - loss: 0.1888 - accuracy: 0.9280 - val_loss: 0.7095 - val_accuracy: 0.7899
Epoch 289/1000
72/72 [==============================] - 3s 45ms/step - loss: 0.1900 - accuracy: 0.9308 - val_loss: 0.6934 - val_accuracy: 0.7778


In [ ]:
train_sub = [2,3,4,5,6,7,8,9]
val_sub = [1]
test_sub = [1]

train_without_aug(train_sub, val_sub, test_sub)